In [10]:
import json
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import requests
import youtube_dl

In [ ]:
spotify_token = "BQDsS3g37DbaXNYqSwziau6FYRy_EFT0lAwR2Vdsw0CzZV6417mJwItmixRANmsaxgmAOWMmMPV1GRYr2aWlsZayP8PPQNPk1jhL4NrlsbZEpa_ZC1sqYsUR2cTrdnKNpIbJxEX2-wmOUIdhoZ-UZciiYro0ewk_aHSrQm7nQm8uiP6TbkSQr6g5U2PZbGyKGf5ZNUZaVBEnDRPLATGg5hjPE8omJjgC"
spotify_user_id = "fv9zz0zcfxemmuvj7ztnclgmx"

class createPlayList:
    
    def __init__(self):
        self.user_id = spotify_user_id
        self.spotify_token = spotify_token
        self.youtube_client = self.get_youtube_client()
        
    
        
    
    def get_youtube_client(self):
        os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
        api_service_name = "youtube"
        api_version = "v3"
        client_secrets_file = "client_secret.json"

        # Get credentials and create an API client
        scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
        credentials = flow.run_console()

        # from the Youtube DATA API
        youtube_client = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)

        return youtube_client
    
    def get_liked_videos(self):
        video_title = item["snippet"]["title"]
        youtube_url = "https://www.youtube.com/watch?v={}".format(item["id"])

        # use youtube_dl to collect the song name & artist name
        video = youtube_dl.YoutubeDL({}).extract_info(youtube_url, download=False)
        song_name = video["track"]
        artist = video["artist"]
            
        request = self.youtube_client.videos().list(
        part="snippet,contentDetails,statistics",
        myRating="like"
        )
        
        response = request.execute()
        
        for item in response["items"]:
            video_title = item["snippet"]["title"]
            youtube_url = "https://www.youtube.com/watch?v={}".format(item["id"])

            # use youtube_dl to collect the song name & artist name
            video = youtube_dl.YoutubeDL({}).extract_info(youtube_url, download=False)
            song_name = video["track"]
            artist = video["artist"]
                
            if song_name is not None and artist is not None:
                self.all_song_info[video_title] = {
                    "youtube_url": youtube_url,
                    "song_name": song_name,
                    "artist": artist,
                    "spotify_uri":self.get_spotify_uri(song_name,artist)
                }
                    
    def create_playlist(self):
        pass
    
        request_body = json.dumps({
            "name":"Youtube Likes",
            "description":"All Youtube liked videos",
            "public":True
        })
        
        query = "https://api.spotify.com/v1/users/{}/playlists".format(spotify_user_id)
        response = requests.post(
            query,
            data=request_body,
            headers={
                "Content-Type":"application/json",
                "Authorization":"Bearer.{}".format(spotify_token)
            }
        )
        
        response_json = response.json()
        
        return response_json["id"]
    
    
    
    def get_spotify_uri(self, song_name, artist):
        query = "https://api.spotify.com/v1/search?query=track%3A{}+artist%3A{}&type=track&offset=0&limit=20".format(
            song_name,
            artist
        )
        
        response = requests.get(
            query,
            headers={
                "Content-Type":"application/json",
                "Authorization":"Bearer.{}".format(spotify_token)
            }
        )
        
        response_json = response.json()
        songs = response_json["tracks"]["items"]
        
        uri = songs[0]["uri"]
        
        return uri
    
    def add_song_to_play_list(self):
        self.get_liked_videos()
        uris = [info["spotify_uri"] for song, info in self.all_song_info.items()]
        playlist_id = self.create_playlist()
        request_data = json.dumps(uris)
        query = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist_id)
        response = response.post(
            query,
            data=request_data,
            headers={
                "Content-Type":"application/json",
                "Authorization":"Bearer.{}".format(spotify_token)
            }
        )
        
        response_json = response.json()
        return response_json
    
if __name__ == '__main__':
    cp = createPlayList()
    cp.add_song_to_playlist
        
    
        
    
    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=NZQ3rkgikUx6RFR644fHWpiYP8HTBV&prompt=consent&access_type=offline
